In [1]:
import pandas as pd
import re
import os

In [2]:
def find_msgs_by_intent_from_nlu(chemin,intent):
    f = open(chemin)
    lignes_pret=[]
    lignes=[]
    for each in f:
        if each.startswith("## intent: {intent_name}".format(intent_name=intent)):
            for each in f :
                if each.startswith("## intent:") :
                    break
                lignes.append(each)
    for element in lignes :
        new_element = element[2:-1]
        lignes_pret.append(new_element)          
    return lignes_pret

In [3]:
find_msgs_by_intent_from_nlu("data\\nlu.md","greet")

['eeeee hello eeeee',
 'eeeeegood eveningeeeee',
 'eeeeegood afternooneeeee',
 'eeeeehey thereeeeee',
 'hey',
 'hello',
 'hi',
 'good morning',
 'good evening',
 'good afternoon',
 'hey there',
 'hello robot',
 'hiiiiiiiiiiiiii',
 'bonjourrrr',
 'salut',
 'beh lkhir',
 'helllllllllo',
 'aaslemaa',
 'donnnne',
 'donnnne2',
 'hayhay',
 'a',
 'a1',
 'a2',
 'a3']

In [4]:
def fetch_intents_from_nlu(path):
    list_intent = []
    list_intent_syn =[]
    fichier = open(r'{}\nlu.md'.format(path), "r")
    for line in fichier:
        if line.startswith("## intent: "):
            substring = line[11:]
            substring = substring.replace("\n","")
            intent_syn = line
            list_intent_syn.append(intent_syn)
            list_intent.append(substring)
    fichier.close()
    return list_intent

In [5]:
fetch_intents_from_nlu("data")

['greet',
 'goodbye',
 'thank you',
 'create task',
 'stream creation',
 'names',
 'org_id',
 'ws_id',
 'stream_id',
 'task_name',
 'task_priority']

In [6]:
def fetch_intents_from_domain(path):
    list_intent = []
    list_intent_syn =[]
    fichier = open(r'{}\domain.yml'.format(path), "r")
    for line in fichier:
        if line.startswith("- ") and not line.startswith("- utter") :
            print(line)
            substring = line[2:]
            substring = substring.replace("\n","")
            intent_syn = line
            list_intent_syn.append(intent_syn)
            list_intent.append(substring)
    fichier.close()
    return list_intent

In [7]:
fetch_intents_from_domain("data")

- task_priority

- task_name

- stream_id

- ws_id

- org_id

- names

- stream creation

- create task

- thank you

- goodbye

- greet



['task_priority',
 'task_name',
 'stream_id',
 'ws_id',
 'org_id',
 'names',
 'stream creation',
 'create task',
 'thank you',
 'goodbye',
 'greet']

In [8]:
def create_rasa_files(path, create_files_path):
    """    Converts an CSV file created with the specified format to RASA accepted nlu.md format
    :param path: path where the CSV file is present
    :param create_files_path: path where the nlu.md file needs to be created\n
    :return: return nothing. A file is created in the path specified via create_files_path\n    """
    msg_to_add = []
    list_csv=[]  
    actions_pos=0
    intent_pos=0
    nb=0
    
    df = pd.read_csv(r"{}".format(path)) 
    file = open(r'{}\nlu.md'.format(create_files_path), "a")
    intents = list(df.columns)    #case when intent does not exist in nlu.md
    for item in intents:  
        if item not in fetch_intents_from_nlu(create_files_path):   
            file.write("## intent: {intent_name}\n".format(intent_name=item))
            for sent in df[item]:
                if type(sent) is str :
                    file.write("- {}\n".format(sent))
    file.close()   
                #case when intent exists in nlu.md : add new messages only in the right position\n   
                #ile = open(r\'{}\\nlu.md\'.format(create_files_path), "a+")
            
            
    for item in intents:
        
        if item in fetch_intents_from_nlu(create_files_path):
            msgs_nlu = find_msgs_by_intent_from_nlu("data\\nlu.md",item)
            
            
            
        print("**************** what exists in nlu")
        print(msgs_nlu)
        for i in df[item] :
            if i not in msgs_nlu and type(i) is str or type(i) is int :
                msg_to_add.append(i)                   
        print("**************** msg to add")
        print(msg_to_add)           
                
        print(item)
        
        my_file = open(r'{}\nlu.md'.format(create_files_path))
        string_list = my_file.readlines()
        for index, value in enumerate(string_list):
            if value == "## intent: {intent_name}\n".format(intent_name=item):
                intent_pos = index
                print(value)
        my_file.close()
        print(intent_pos)
        if msg_to_add != [] :
            
            my_file = open(r'{}\nlu.md'.format(create_files_path), "w")
            for index, value in enumerate(msg_to_add):
                string_list.insert(intent_pos+1+index , '- {new_msg}\n'.format(new_msg=value))
            new_file_contents = "".join(string_list)
            my_file.write(new_file_contents)
            msg_to_add = []
            my_file.close()
        
                             
    file = open(r'{}\domain.yml'.format(create_files_path), "w")
    
    if os.stat("data\\domain.yml").st_size == 0 :
        file.write("intents:\n")
        file.write("actions:\n")
        
    file.close()
                                                                                
    nb=0                                                                            
    for intent_name in intents:
        
        if intent_name not in fetch_intents_from_domain(create_files_path) :
            nb=nb+1
            file = open(r'{}\domain.yml'.format(create_files_path))
            string_list = file.readlines()
            for index, value in enumerate(string_list):
                if value == "intents:\n":
                    intent_pos = index
                if value == "actions:\n":
                    actions_pos = index
            my_file.close()
            my_file = open(r'{}\domain.yml'.format(create_files_path), "w") 
            string_list.insert(intent_pos+1 ,'- {new_msg}\n'.format(new_msg=intent_name))
            string_list.insert(actions_pos+1+nb , "- utter_{}\n".format(intent_name))
            new_file_contents = "".join(string_list)
            my_file.write(new_file_contents)
            my_file.close()

In [9]:
create_rasa_files("greet.csv","data")

**************** what exists in nlu
['eeeee hello eeeee', 'eeeeegood eveningeeeee', 'eeeeegood afternooneeeee', 'eeeeehey thereeeeee', 'hey', 'hello', 'hi', 'good morning', 'good evening', 'good afternoon', 'hey there', 'hello robot', 'hiiiiiiiiiiiiii', 'bonjourrrr', 'salut', 'beh lkhir', 'helllllllllo', 'aaslemaa', 'donnnne', 'donnnne2', 'hayhay', 'a', 'a1', 'a2', 'a3']
**************** msg to add
[]
greet
## intent: greet

0
**************** what exists in nlu
['bye', 'ciaoo', 'guten morgen', 'see you later', 'great bye bye', 'thank you bye', 'goodnight', 'good bye', 'byyyyye', 'a +', 'byby', 'byyyyyyyyyyyye', 'byebyee', 'seyes rouhek', 'b', 'b1', 'b2', 'b3']
**************** msg to add
[]
goodbye
## intent: goodbye

26
**************** what exists in nlu
['merciiii', 'okay cool uh thank you', 'okay thank you', 'you rock', 'and thats all thank you', 'sorry about my mistakes thank you', 'meera thank you', 'thank you for the opportunity to work with you. I appreciate it very much', 'am